A brief theory review:
\begin{align}
V \equiv v \, e^{i\,\theta}\,,\\
S \equiv p + i\,q\,,
\end{align}

then
\begin{equation}
S = V \cdot (YV)^*\equiv V\cdot I^*\\
\end{equation}
Rearanging so that
\begin{equation}
V = \left[\begin{array}{c}V_{\rm obs}\\ V_{\rm nobs} \end{array}\right]\,,
I = \left[\begin{array}{c}I_{\rm obs}\\ I_{\rm nobs} \end{array}\right]\,,
S = \left[\begin{array}{c}S_{\rm obs}\\ S_{\rm nobs} \end{array}\right]\,,
\end{equation}
then

\begin{equation}
\left[\begin{array}{c}I_{\rm obs}\\ I_{\rm nobs} \end{array}\right] = \left[\begin{array}{cc} Y_{\rm obs} &\tilde Y\\  \tilde Y^\top & Y_{\rm nobs} \end{array}\right] \left[\begin{array}{c}V_{\rm obs}\\ V_{\rm nobs} \end{array}\right]
\end{equation}

\begin{equation}
\underbrace{I_{\rm obs} - \tilde Y\, Y_{\rm nobs}^{-1}\, I_{\rm obs}}_{\equiv I_{\rm r}} = \underbrace{\Big(Y_{\rm obs} - \tilde Y\, Y_{\rm nobs}^{-1}\, \tilde Y^\top\Big)}_{\equiv Y_{\rm r}}V_{\rm obs}
\end{equation}

\begin{equation}
S_{\rm r} \equiv V_{\rm obs}\cdot I_{\rm r} = V_{\rm obs}\cdot (Y_{\rm r}\,V_{\rm obs})^* 
\end{equation}
And, finally,
\begin{equation}
\underbrace{S_{\rm obs}}_\text{what is observed} = \underbrace{V_{\rm obs}\cdot (Y_{\rm r}\,V_{\rm obs})^*}_\text{what the PIML learns} + \underbrace{V_{\rm obs}\cdot (\tilde Y Y_{\rm nobs}^{-1} I_{\rm nobs})^*}_\text{what the NN learns}
\end{equation}

In [ ]:
using Pkg
Pkg.activate("..")
using PIML4Power
using Flux
using Plots

In [ ]:
# import data
data, mat, id = load_data("../data/test_data_set.h5");
id_batch = collect(1:5:200)
opt = ADAM(0.01)

In [ ]:
# perform the Kron reduction of the system
# (will be use for comparison)
Y = build_admittance_matrix(data.b, data.g, data.bsh,
    data.gsh, data.epsilon)

y_r, ysh_r, epsilon_r = kron_reduction(Y, id.pv, id.pq, alpha = 1E-2)
id_r = PIML4Power.create_indices(id.pv[1], collect(1:length(id.pv)),
    length(id.pv), epsilon_r)

mat_r = PIML4Power.create_incidence_matrices(id_r);

red_data = PIML4Power.SystemData(data.v[id.pv, :], data.th[id.pv, :], data.p[id.pv, :],
    data.q[id.pv, :], epsilon_r, imag(y_r), real(y_r), imag(ysh_r), real(ysh_r));

In [ ]:
# initialize the grid parameters
Nline = size(id_r.epsilon, 1)
gamma = 2 * ones(Nline)
beta = 4 * ones(Nline)
gsh = 1E-1 * ones(id_r.Nbus)
bsh = 1E-1 * ones(id_r.Nbus);

In [ ]:
# create a NN    
nn = Chain(Dense(2*id_r.Nbus, 100, tanh),
    Dense(100, 100, tanh),
    Dense(100, 100, tanh),
    Dense(100, 100, tanh),
    Dense(100, 2*id_r.Nbus));

In [ ]:
# train the model
Nepoch = 50000
Ninter = 1000
reg = 10
logs = train_hybrid_V2S_map!(red_data, mat_r, id_r, id_batch, nn, opt,
    PIML4Power.exp_param, beta, gamma, bsh, gsh, Ninter = Ninter,
    Nepoch = Nepoch, reg = reg);

In [ ]:
# compute contribution of non-observed buses
obs = id.pv
nobs = setdiff(1:id.Nbus, obs)

x = nn([data.v[obs,:]; data.th[obs,:]])

p_est, q_est = PIML4Power.contribution_of_nobs_buses(Y, obs, nobs, data.th, data.v, data.p, data.q)

In [ ]:
plot(p_est[:,end])
plot!(x[1:id_r.Nbus,end])